# Spotify popularity calculator

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from fuzzywuzzy import fuzz
import pandas as pd
import json


In [ ]:
# fetch the csv file
df_csv = pd.read_csv('/Users/ntorqulu/PycharmProjects/pythonProject/assets/msd_summary.csv')
# convert csv col author and title to dict
author_csv = {"artist": df_csv['artist_name'].tolist()}
title_csv = {"title": df_csv['title'].tolist()}
# merge dicts
songs_name_author = {**title_csv, **author_csv}
print(songs_name_author)
# print(songs_name_author)

In [ ]:
# open spotify account connection
scope = "playlist-modify-public"
username = "fucyhdjvp8889yi08rjxwatno"
token = SpotifyOAuth(scope=scope, username=username)
spotifyObject = spotipy.Spotify(auth_manager=token)

# create the playlist
playlist_name = input("Enter a playlist name: ")
playlist_description = input("Enter a playlist description: ")

# create empty playlist
spotifyObject.user_playlist_create(user=username, name=playlist_name, public=True, description=playlist_description)

list_of_songs = []
# for each title in the csv col title
for i in range(2000, 3000):
    # reset the values
    max_value = 0
    curr_value = 0
    step = 0
    enter = 1
    print("here")
    # search a matching track in spotify, fetch only first five results
    results = spotifyObject.search(q=f"{songs_name_author['title'][i]} {songs_name_author['artist'][i]}", limit=5,
                                   type='track')  # get 5 responses since first isn't always accurate
    # if there aren't coincidences
    if results['tracks']['total'] == 0:  # if track isn't on spotify as queried
        # search for the title, assume similar music taste...
        results = spotifyObject.search(q=f"{songs_name_author['title'][i]}", limit=5, type='track')
        # if there isn't any matching title
        if results['tracks']['total'] == 0:
            # search for the author, assume similar music taste...
            results = spotifyObject.search(q=f"{songs_name_author['artist'][i]}", limit=5, type='track')
            if results['tracks']['total'] == 0:
                enter = 0
                print("NOT FOUND: " + songs_name_author['title'][i] + songs_name_author['artist'][i])
                # exit if no coincidences

    # check again results after the if-nested-conditions
    if enter:
        # for each query returned from spotify, study max 5 possible matching tracks
        for j in range(len(results['tracks']['items'])):
            # if the substrings are similar ( > 90 )
            if fuzz.partial_ratio(results['tracks']['items'][j]['name'], songs_name_author['title'][i]) > 90\
                    and fuzz.partial_ratio(results['tracks']['items'][j]['artists'][0]['name'], songs_name_author['artist'][i]) > 90:  # get
                # right response by matching on artist and title
                # we keep this track!
                list_of_songs.append(results['tracks']['items'][j]['id'])  # append track id
                print("Song " + results['tracks']['items'][j]['name'] + "added")
                curr_value = -1
                break  # don't want repeats of a sample ex: different versions
            else:
                # if the titles aren't matching, keep the similarity number
                curr_value = fuzz.partial_ratio(results['tracks']['items'][j]['artists'][0]['name'],
                                                songs_name_author['artist'][i]) \
                             + fuzz.partial_ratio(results['tracks']['items'][j]['name'], songs_name_author['title'][i])
                # and make it the biggest possible from the five tracks
                if curr_value > max_value:
                    max_value = curr_value
                    step = j  # and keep this track!
        if curr_value != -1:
            # if the title isn't matching we keep the track with the highest score
            list_of_songs.append(results['tracks']['items'][step]['id'])  # append track id
            print("Song " + results['tracks']['items'][step]['name'] + "added")

prePlaylist = spotifyObject.user_playlists(user=username)
playlist = prePlaylist["items"][0]["id"]

# append the songs to the playlist
while list_of_songs:
    spotifyObject.user_playlist_add_tracks(username, playlist, list_of_songs[:100], position=None)
    list_of_songs = list_of_songs[100:]


In [ ]:
def data_scraping():
    credentials = json.load(open('Authorization.json'))
    # own client id and client secret on the Spotify developers account
    client_id = credentials['client_id']
    client_secret = credentials['client_secret']

    client_credentials_manager = SpotifyOAuth(client_id=client_id, client_secret=client_secret)

    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    uri = "spotify:user:Test_1:playlist:1gSqxxi07KAKHd6aRo2QzE"  # the URI is split by ':' to get the username and playlist ID
    username = uri.split(':')[2]
    playlist_id = uri.split(':')[4]

    results = sp.user_playlist_tracks(username, playlist_id)

    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    results = tracks

    for i in range(len(results)):
        print(i)  # counter
        if i == 0:
            playlist_tracks_id = results[i]['track']['id']
            playlist_tracks_titles = results[i]['track']['name']
            playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
            playlist_tracks_popularity = results[i]['track']['popularity']

            artist_list = []
            for artist in results[i]['track']['artists']:
                artist_list = artist['name']
            playlist_tracks_artists = artist_list

            features = sp.audio_features(playlist_tracks_id)
            features_df = pd.DataFrame(data=features, columns=features[0].keys())
            features_df['title'] = playlist_tracks_titles
            features_df['all_artists'] = playlist_tracks_artists
            features_df['popularity'] = playlist_tracks_popularity
            features_df = features_df[['title', 'all_artists', 'popularity']]
            continue
        else:
            try:
                playlist_tracks_titles = results[i]['track']['name']
                playlist_tracks_popularity = results[i]['track']['popularity']
                artist_list = []
                for artist in results[i]['track']['artists']:
                    artist_list = artist['name']
                playlist_tracks_artists = artist_list
                new_row = {'title': [playlist_tracks_titles],
                           'all_artists': [playlist_tracks_artists],
                           'popularity': [playlist_tracks_popularity],
                           }

                dfs = [features_df, pd.DataFrame(new_row)]
                features_df = pd.concat(dfs, ignore_index=True)
            except:
                continue

    return features_df


music_data = data_scraping()
music_data.to_csv("csv_" + str("song_views") + ".csv", encoding='utf-8', index="false")
